In [1]:
import sys
import numpy as np
import geopy.distance
from math import radians, cos, sin, asin, sqrt

In [2]:
city_name = []
city_latitude = []
city_longitude = []

with open('city-gps.txt') as file:
    for line in file:
        l = line.split()
        city_name.append(l[0])
        city_latitude.append(l[1])
        city_longitude.append(l[2])
        
city_gps = list(zip(city_name, city_latitude, city_longitude))

In [3]:
first_city = []
second_city = []
highway_length = []
speed_limit = []
highway_name = []


with open('road-segments.txt') as file:
    for line in file:
        l = line.split()
        first_city.append(l[0])
        second_city.append(l[1])
        highway_length.append(l[2])
        speed_limit.append(l[3])
        highway_name.append(l[4])
        
road_segments = list(zip(first_city, second_city, highway_length, speed_limit, highway_name))

In [16]:
road_segments

[('"Y"_City,_Arkansas', 'Acorn,_Arkansas', '15', '45', 'US_71/270'),
 ('"Y"_City,_Arkansas', 'Greenwood,_Arkansas', '46', '45', 'US_71'),
 ('"Y"_City,_Arkansas', 'Hot_Springs,_Arkansas', '70', '45', 'US_270'),
 ('Abbot_Village,_Maine', 'Bingham,_Maine', '24', '45', 'ME_16'),
 ('Abbot_Village,_Maine', 'Guilford,_Maine', '4', '45', 'ME_6/15/16'),
 ('Abbot_Village,_Maine', 'Jackman_Station,_Maine', '73', '45', 'ME_6/15'),
 ('Abbotsford,_Wisconsin',
  'Jct_WI_29_&_WI_97,_Wisconsin',
  '12',
  '45',
  'WI_29'),
 ('Abbotsford,_Wisconsin', 'Marshfield,_Wisconsin', '22', '45', 'WI_13'),
 ('Abbotsford,_Wisconsin', 'Medford,_Wisconsin', '14', '45', 'WI_13'),
 ('Abbotsford,_Wisconsin', 'Withee,_Wisconsin', '14', '45', 'WI_29'),
 ('Abbott,_New_Mexico', 'Clayton,_New_Mexico', '63', '45', 'US_56'),
 ('Abbott,_New_Mexico', 'Roy,_New_Mexico', '26', '45', 'NM_39'),
 ('Abbott,_New_Mexico', 'Springer,_New_Mexico', '20', '45', 'US_56'),
 ('Abbyville,_Kansas', 'Arlington,_Kansas', '5', '45', 'KS_14'),
 ('A

In [5]:
def haversine_distance(start_city, end_city): # heuristic function
    
    start_lat, start_long, end_lat, end_long = 0, 0, 0, 0

    for i in range(len(city_gps)):
        
        if city_gps[i][0] == start_city:
            start_lat = city_gps[i][1]
            start_long = city_gps[i][2]
            
        if city_gps[i][0] == end_city:
            end_lat = city_gps[i][1]
            end_long = city_gps[i][2]
            
    coord1 = (start_lat, start_long)
    coord2 = (end_lat, end_long)
    
    distance = round(geopy.distance.vincenty(coord1, coord2).miles,2)
    
    return distance
            

In [14]:
def heu_distance(start_city, end_city):
    
    start_lat, start_long, end_lat, end_long = 0, 0, 0, 0
    
    for i in range(len(city_gps)):
        
        if city_gps[i][0] == start_city:
            start_lat = radians(float(city_gps[i][1]))
            start_long = radians(float(city_gps[i][2]))
            
        if city_gps[i][0] == end_city:
            end_lat = radians(float(city_gps[i][1]))
            end_long = radians(float(city_gps[i][2]))
            
    diff_lat = end_lat - start_lat
    diff_long = end_long - start_long
    
    a = sin(diff_lat/2)**2 + cos(start_lat) * cos(end_lat) * sin(diff_long/2)**2
    
    c = 2 * asin(sqrt(a))
    
    r = 3956
    
    return round((c * r),2)

In [7]:
print(city_gps[0], city_gps[1])

('"Y"_City,_Arkansas', '34.735104', '-94.073257') ('Abbot_Village,_Maine', '45.1976844', '-69.458819')


In [11]:
haversine_distance('"Y"_City,_Arkansas', 'Abbot_Village,_Maine')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


1484.2

In [15]:
heu_distance('"Y"_City,_Arkansas', 'Abbot_Village,_Maine')

1480.85

In [18]:
# code to generate successor of each node
def city_successor(start_city):
    
    city_successor = []
    
    for i in range(len(road_segments)):
        if road_segments[i][0] == start_city:
            city_successor.append(road_segments[i][1])
            
    return city_successor
    

In [ ]:
def get_route(start, end, cost):
    
    """
    Find shortest driving route between start city and end city
    based on a cost function.

    1. Your function should return a dictionary having the following keys:
        -"route-taken" : a list of pairs of the form (next-stop, segment-info), where
           next-stop is a string giving the next stop in the route, and segment-info is a free-form
           string containing information about the segment that will be displayed to the user.
           (segment-info is not inspected by the automatic testing program).
        -"total-segments": an integer indicating number of segments in the route-taken
        -"total-miles": a float indicating total number of miles in the route-taken
        -"total-hours": a float indicating total amount of time in the route-taken
        -"total-delivery-hours": a float indicating the expected (average) time 
                                   it will take a delivery driver who may need to return to get a new package
    2. Do not add any extra parameters to the get_route() function, or it will break our grading and testing code.
    3. Please do not use any global variables, as it may cause the testing code to fail.
    4. You can assume that all test cases will be solvable.
    5. The current code just returns a dummy solution.
    """

    route_taken = [("Martinsville,_Indiana","IN_37 for 19 miles"),
                   ("Jct_I-465_&_IN_37_S,_Indiana","IN_37 for 25 miles"),
                   ("Indianapolis,_Indiana","IN_37 for 7 miles")]
    
    return {"total-segments" : len(route_taken), 
            "total-miles" : 51., 
            "total-hours" : 1.07949, 
            "total-delivery-hours" : 1.1364, 
            "route-taken" : route_taken}

In [ ]:
# Please don't modify anything below this line
#
if __name__ == "__main__":
    if len(sys.argv) != 4:
        raise(Exception("Error: expected 3 arguments"))

    (_, start_city, end_city, cost_function) = sys.argv
    if cost_function not in ("segments", "distance", "time", "delivery"):
        raise(Exception("Error: invalid cost function"))

    result = get_route(start_city, end_city, cost_function)

    # Pretty print the route
    print("Start in %s" % start_city)
    for step in result["route-taken"]:
        print("   Then go to %s via %s" % step)

    print("\n          Total segments: %4d" % result["total-segments"])
    print("             Total miles: %8.3f" % result["total-miles"])
    print("             Total hours: %8.3f" % result["total-hours"])
    print("Total hours for delivery: %8.3f" % result["total-delivery-hours"])

